In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from glob import glob
import xml.etree.ElementTree as xet

total_files = 0
total_folders = 0

# First pass to count total files and folders
for dirname, _, filenames in os.walk('/kaggle/input'):
    total_folders += 1
    total_files += len(filenames)

print(f"Total number of folders: {total_folders}")
print(f"Total number of files: {total_files}")
print("=" * 50)

# Second pass to print folder details
for dirname, _, filenames in os.walk('/kaggle/input'):
    if filenames:
        print(f"Folder: {dirname}")
        print(f"Number of files: {len(filenames)}")
        print(f"Sample file: {os.path.join(dirname, filenames[0])}")
        print("-" * 50)

Total number of folders: 40
Total number of files: 3396
Folder: /kaggle/input/indian-vehicle-dataset/google_images
Number of files: 883
Sample file: /kaggle/input/indian-vehicle-dataset/google_images/2302b12c-3d74-4d6c-b376-ba8ab308a8c3___big_215451_1320738450.jpg.jpeg
--------------------------------------------------
Folder: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/HP
Number of files: 44
Sample file: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/HP/HP19.xml
--------------------------------------------------
Folder: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/AP
Number of files: 74
Sample file: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/AP/AP20.xml
--------------------------------------------------
Folder: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/PY
Number of files: 34
Sample file: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/PY/PY16.xml
--------------------------------------------------
Folder: /kaggle/input/indian-vehicle-dataset/S

In [2]:
import torch

if torch.cuda.is_available():
    print("Using GPU:", torch.cuda.get_device_name(0))
elif 'COLAB_TPU_ADDR' in os.environ:
    print("Using TPU")
else:
    print("No GPU or TPU found.")

Using GPU: Tesla T4


In [3]:
!nvidia-smi

Sat Feb 22 04:57:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 14.9 MB/s eta 0:00:00a 0:00:01


In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11s.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 18.4M/18.4M [00:00<00:00, 167MB/s]


In [6]:
path = glob('/kaggle/input/indian-vehicle-dataset/google_images/*.xml')
print(len(path))


labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[], plate_number=[])

for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    plate_number = str(member_object.find('name').text)
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)
    labels_dict['plate_number'].append(plate_number)

440


In [7]:
new_paths = glob('/kaggle/input/indian-vehicle-dataset/State-wise_OLX/*/*.xml')
print(len(new_paths))


new_labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[], plate_number=[])

for filename in new_paths:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    plate_number = str(member_object.find('name').text)
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    new_labels_dict['filepath'].append(filename)
    new_labels_dict['xmin'].append(xmin)
    new_labels_dict['xmax'].append(xmax)
    new_labels_dict['ymin'].append(ymin)
    new_labels_dict['ymax'].append(ymax)
    new_labels_dict['plate_number'].append(plate_number)

603


In [8]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax,plate_number
0,/kaggle/input/indian-vehicle-dataset/google_im...,29,161,265,325,MH06AW8929
1,/kaggle/input/indian-vehicle-dataset/google_im...,144,369,82,230,DL1N4268
2,/kaggle/input/indian-vehicle-dataset/google_im...,254,472,516,570,MH03BS7778
3,/kaggle/input/indian-vehicle-dataset/google_im...,267,500,511,574,TN19TC91
4,/kaggle/input/indian-vehicle-dataset/google_im...,231,434,82,156,KL01CC50


In [9]:
new_df = pd.DataFrame(new_labels_dict)
new_df.to_csv('new_labels.csv',index=False)
new_df.head()

,filepath,xmin,xmax,ymin,ymax,plate_number
0,/kaggle/input/indian-vehicle-dataset/State-wis...,105,167,183,197,HP54C6564
1,/kaggle/input/indian-vehicle-dataset/State-wis...,81,131,124,137,HP302165
2,/kaggle/input/indian-vehicle-dataset/State-wis...,30,74,97,111,HP01A7630
3,/kaggle/input/indian-vehicle-dataset/State-wis...,102,165,352,368,HP52B5958
4,/kaggle/input/indian-vehicle-dataset/State-wis...,86,160,194,218,HP23A6365


In [10]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'/kaggle/input/indian-vehicle-dataset/google_images/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)
df.head()

,filepath,xmin,xmax,ymin,ymax,plate_number,filename,width,height
0,/kaggle/input/indian-vehicle-dataset/google_im...,29,161,265,325,MH06AW8929,/kaggle/input/indian-vehicle-dataset/google_im...,500,409
1,/kaggle/input/indian-vehicle-dataset/google_im...,144,369,82,230,DL1N4268,/kaggle/input/indian-vehicle-dataset/google_im...,509,290
2,/kaggle/input/indian-vehicle-dataset/google_im...,254,472,516,570,MH03BS7778,/kaggle/input/indian-vehicle-dataset/google_im...,768,1024
3,/kaggle/input/indian-vehicle-dataset/google_im...,267,500,511,574,TN19TC91,/kaggle/input/indian-vehicle-dataset/google_im...,1024,706
4,/kaggle/input/indian-vehicle-dataset/google_im...,231,434,82,156,KL01CC50,/kaggle/input/indian-vehicle-dataset/google_im...,500,234


In [11]:
# parsing
def newparsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'/kaggle/input/indian-vehicle-dataset/State-wise_OLX/{name[:2]}/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

new_df[['filename','width','height']] = new_df['filepath'].apply(newparsing).apply(pd.Series)
new_df.head()

,filepath,xmin,xmax,ymin,ymax,plate_number,filename,width,height
0,/kaggle/input/indian-vehicle-dataset/State-wis...,105,167,183,197,HP54C6564,/kaggle/input/indian-vehicle-dataset/State-wis...,272,306
1,/kaggle/input/indian-vehicle-dataset/State-wis...,81,131,124,137,HP302165,/kaggle/input/indian-vehicle-dataset/State-wis...,272,204
2,/kaggle/input/indian-vehicle-dataset/State-wis...,30,74,97,111,HP01A7630,/kaggle/input/indian-vehicle-dataset/State-wis...,191,139
3,/kaggle/input/indian-vehicle-dataset/State-wis...,102,165,352,368,HP52B5958,/kaggle/input/indian-vehicle-dataset/State-wis...,272,590
4,/kaggle/input/indian-vehicle-dataset/State-wis...,86,160,194,218,HP23A6365,/kaggle/input/indian-vehicle-dataset/State-wis...,272,297


In [12]:
# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
df.head()

,filepath,xmin,xmax,ymin,ymax,plate_number,filename,width,height,center_x,center_y,bb_width,bb_height
0,/kaggle/input/indian-vehicle-dataset/google_im...,29,161,265,325,MH06AW8929,/kaggle/input/indian-vehicle-dataset/google_im...,500,409,0.190000,0.721271,0.264000,0.146699
1,/kaggle/input/indian-vehicle-dataset/google_im...,144,369,82,230,DL1N4268,/kaggle/input/indian-vehicle-dataset/google_im...,509,290,0.503929,0.537931,0.442043,0.510345
2,/kaggle/input/indian-vehicle-dataset/google_im...,254,472,516,570,MH03BS7778,/kaggle/input/indian-vehicle-dataset/google_im...,768,1024,0.472656,0.530273,0.283854,0.052734
3,/kaggle/input/indian-vehicle-dataset/google_im...,267,500,511,574,TN19TC91,/kaggle/input/indian-vehicle-dataset/google_im...,1024,706,0.374512,0.768414,0.227539,0.089235
4,/kaggle/input/indian-vehicle-dataset/google_im...,231,434,82,156,KL01CC50,/kaggle/input/indian-vehicle-dataset/google_im...,500,234,0.665000,0.508547,0.406000,0.316239


In [13]:
# center_x, center_y, width , height
new_df['center_x'] = (new_df['xmax'] + new_df['xmin'])/(2*new_df['width'])
new_df['center_y'] = (new_df['ymax'] + new_df['ymin'])/(2*new_df['height'])

new_df['bb_width'] = (new_df['xmax'] - new_df['xmin'])/new_df['width']
new_df['bb_height'] = (new_df['ymax'] - new_df['ymin'])/new_df['height']
new_df.head()

,filepath,xmin,xmax,ymin,ymax,plate_number,filename,width,height,center_x,center_y,bb_width,bb_height
0,/kaggle/input/indian-vehicle-dataset/State-wis...,105,167,183,197,HP54C6564,/kaggle/input/indian-vehicle-dataset/State-wis...,272,306,0.500000,0.620915,0.227941,0.045752
1,/kaggle/input/indian-vehicle-dataset/State-wis...,81,131,124,137,HP302165,/kaggle/input/indian-vehicle-dataset/State-wis...,272,204,0.389706,0.639706,0.183824,0.063725
2,/kaggle/input/indian-vehicle-dataset/State-wis...,30,74,97,111,HP01A7630,/kaggle/input/indian-vehicle-dataset/State-wis...,191,139,0.272251,0.748201,0.230366,0.100719
3,/kaggle/input/indian-vehicle-dataset/State-wis...,102,165,352,368,HP52B5958,/kaggle/input/indian-vehicle-dataset/State-wis...,272,590,0.490809,0.610169,0.231618,0.027119
4,/kaggle/input/indian-vehicle-dataset/State-wis...,86,160,194,218,HP23A6365,/kaggle/input/indian-vehicle-dataset/State-wis...,272,297,0.452206,0.693603,0.272059,0.080808


In [14]:
data = pd.concat([df,new_df]).sample(frac=1).reset_index(drop=True)
data.head()

,filepath,xmin,xmax,ymin,ymax,plate_number,filename,width,height,center_x,center_y,bb_width,bb_height
0,/kaggle/input/indian-vehicle-dataset/google_im...,52,401,87,175,TN66U8215,/kaggle/input/indian-vehicle-dataset/google_im...,498,313,0.454819,0.418530,0.700803,0.281150
1,/kaggle/input/indian-vehicle-dataset/google_im...,387,592,395,442,TN58AM1,/kaggle/input/indian-vehicle-dataset/google_im...,960,720,0.509896,0.581250,0.213542,0.065278
2,/kaggle/input/indian-vehicle-dataset/State-wis...,116,175,117,130,GJ01RS0198,/kaggle/input/indian-vehicle-dataset/State-wis...,272,204,0.534926,0.605392,0.216912,0.063725
3,/kaggle/input/indian-vehicle-dataset/State-wis...,102,165,352,368,HP52B5958,/kaggle/input/indian-vehicle-dataset/State-wis...,272,590,0.490809,0.610169,0.231618,0.027119
4,/kaggle/input/indian-vehicle-dataset/State-wis...,143,208,228,246,TR02L0569,/kaggle/input/indian-vehicle-dataset/State-wis...,272,363,0.645221,0.652893,0.238971,0.049587


In [15]:
data.count()

filepath        1043
xmin            1043
xmax            1043
ymin            1043
ymax            1043
plate_number    1043
filename        1043
width           1043
height          1043
center_x        1043
center_y        1043
bb_width        1043
bb_height       1043
dtype: int64

In [16]:
df_metadata = pd.DataFrame()
df_metadata = data.rename(columns={"filename": "image_path"})
df_metadata.head()

# Extract individual bbox coordinates
df_metadata["x1_bbox"] = df_metadata["xmin"] 
df_metadata["y1_bbox"] = df_metadata["ymin"] 
df_metadata["x2_bbox"] = df_metadata["xmax"] 
df_metadata["y2_bbox"] = df_metadata["ymax"]
df_metadata["xmid"] = df_metadata["center_x"]
df_metadata["ymid"] = df_metadata["center_y"]
df_metadata = df_metadata.drop(columns=["filepath", "xmin", "xmax", "ymin", "ymax", "width", "height", "center_x", "center_y"])
df_metadata.head()

,plate_number,image_path,bb_width,bb_height,x1_bbox,y1_bbox,x2_bbox,y2_bbox,xmid,ymid
0,TN66U8215,/kaggle/input/indian-vehicle-dataset/google_im...,0.700803,0.281150,52,87,401,175,0.454819,0.418530
1,TN58AM1,/kaggle/input/indian-vehicle-dataset/google_im...,0.213542,0.065278,387,395,592,442,0.509896,0.581250
2,GJ01RS0198,/kaggle/input/indian-vehicle-dataset/State-wis...,0.216912,0.063725,116,117,175,130,0.534926,0.605392
3,HP52B5958,/kaggle/input/indian-vehicle-dataset/State-wis...,0.231618,0.027119,102,352,165,368,0.490809,0.610169
4,TR02L0569,/kaggle/input/indian-vehicle-dataset/State-wis...,0.238971,0.049587,143,228,208,246,0.645221,0.652893


In [17]:
### split the data into train and test
df_train = data.iloc[:800]
df_test = data.iloc[850:]
df_val = data.iloc[800:850]

In [18]:
!mkdir "/kaggle/working/ultralytics"
!mkdir "/kaggle/working/ultralytics/datasets"
!mkdir "/kaggle/working/ultralytics/datasets/train"
!mkdir "/kaggle/working/ultralytics/datasets/val"
!mkdir "/kaggle/working/ultralytics/datasets/test"

In [19]:
import shutil
import os

train_folder = '/kaggle/working/ultralytics/datasets/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    try:
        image_name = os.path.split(fname)[-1]
        txt_name = os.path.splitext(image_name)[0]

        dst_image_path = os.path.join(train_folder,image_name)
        dst_label_file = os.path.join(train_folder,txt_name+'.txt')

        # copy each image into the folder
        if os.path.exists(fname):
            shutil.copy(fname, dst_image_path)
        else:
            raise FileNotFoundError(f"File not found: {fname}")

        # generate .txt which has label info
        label_txt = f'0 {x} {y} {w} {h}'
        with open(dst_label_file,mode='w') as f:
            f.write(label_txt)

            f.close()
    except FileNotFoundError as e:
        print(f"File error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e} with file {fname}")

File error: File not found: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/NL/NL1.jpg
File error: File not found: /kaggle/input/indian-vehicle-dataset/State-wise_OLX/MH/MH5.jpg


In [20]:
df_test.head()

,filepath,xmin,xmax,ymin,ymax,plate_number,filename,width,height,center_x,center_y,bb_width,bb_height
850,/kaggle/input/indian-vehicle-dataset/google_im...,214,425,124,167,MH20CS1941,/kaggle/input/indian-vehicle-dataset/google_im...,616,360,0.518669,0.404167,0.342532,0.119444
851,/kaggle/input/indian-vehicle-dataset/State-wis...,91,179,231,252,UP32CX3400,/kaggle/input/indian-vehicle-dataset/State-wis...,272,363,0.496324,0.665289,0.323529,0.057851
852,/kaggle/input/indian-vehicle-dataset/google_im...,168,905,128,314,MH12DE1433,/kaggle/input/indian-vehicle-dataset/google_im...,1016,555,0.528051,0.398198,0.725394,0.335135
853,/kaggle/input/indian-vehicle-dataset/google_im...,83,242,25,64,TN99F2378,/kaggle/input/indian-vehicle-dataset/google_im...,326,224,0.498466,0.198661,0.487730,0.174107
854,/kaggle/input/indian-vehicle-dataset/State-wis...,85,157,198,215,HP64B3380,/kaggle/input/indian-vehicle-dataset/State-wis...,272,363,0.444853,0.568871,0.264706,0.046832


In [21]:
test_folder = '/kaggle/working/ultralytics/datasets/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values

for fname, x, y, w, h in values:
    try:
        #print(f"Processing: {fname}")
        image_name = os.path.basename(fname)
        txt_name = os.path.splitext(image_name)[0]
        

        dst_image_path = os.path.join(test_folder, image_name)
        dst_label_file = os.path.join(test_folder, txt_name + '.txt')

        if os.path.exists(fname):
            shutil.copy(fname, dst_image_path)
        else:
            raise FileNotFoundError(f"File not found: {fname}")

        label_txt = f'0 {x} {y} {w} {h}'
        with open(dst_label_file, mode='w') as f:
            f.write(label_txt)

    except FileNotFoundError as e:
        print(f"File error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e} with file {fname}")

In [22]:

val_folder = '/kaggle/working/ultralytics/datasets/val'

values = df_val[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    try:
        image_name = os.path.split(fname)[-1]
        txt_name = os.path.splitext(image_name)[0]

        dst_image_path = os.path.join(val_folder,image_name)
        dst_label_file = os.path.join(val_folder,txt_name+'.txt')

        if os.path.exists(fname):
            shutil.copy(fname, dst_image_path)
        else:
            raise FileNotFoundError(f"File not found: {fname}")

        # generate .txt which has label info
        label_txt = f'0 {x} {y} {w} {h}'
        with open(dst_label_file,mode='w') as f:
            f.write(label_txt)

            f.close()
    except FileNotFoundError as e:
        print(f"File error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e} with file {fname}")

In [23]:
%%writefile /kaggle/working/ultralytics/anpr_dataset.yaml
train: /kaggle/working/ultralytics/datasets/train
val: /kaggle/working/ultralytics/datasets/val
test: /kaggle/working/ultralytics/datasets/test
    
nc: 1
    
names: [
    'license_plate'
]

Writing /kaggle/working/ultralytics/anpr_dataset.yaml


In [ ]:
#!wandb disabled
#!yolo task=detect mode=train epochs=200 batch=16 plots=True model="/kaggle/working/yolo11m.pt" data="/kaggle/working/ultralytics/anpr_dataset.yaml" optimizer=AdamW lr0=0.005

In [24]:
!wandb disabled
!yolo task=detect mode=train epochs=200 batch=16 imgsz=768 half=True cache=False \
  model="/kaggle/working/yolo11s.pt" data="/kaggle/working/ultralytics/anpr_dataset.yaml" \
  optimizer=AdamW lr0=0.005 conf=0.4 iou=0.55 \
  perspective=0.003 degrees=30 shear=5.0 \
  plots=True

W&B disabled.
Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/yolo11s.pt, data=/kaggle/working/ultralytics/anpr_dataset.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=768, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.4, iou=0.55, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

In [ ]:
!ls -lrt /kaggle/working/ultralytics/datasets/val/

In [ ]:
!yolo task=detect mode=predict conf=0.50 save=True model=/kaggle/working/runs/detect/train2/weights/best.pt source=/kaggle/working/ultralytics/datasets/val/PB19.jpg

In [ ]:
!ls -lrt /kaggle/working/runs/detect/predict3/

In [ ]:
from IPython.display import Image, display

image_path = '/kaggle/working/runs/detect/predict2/PB19.jpg'

display(Image(filename=image_path, width=600))

In [ ]:
from IPython.display import Image, display

image_path = '/kaggle/working/runs/detect/predict2/PB14.jpg'

display(Image(filename=image_path, width=600))

In [ ]:
from IPython.display import Image, display

image_path = '/kaggle/working/runs/detect/predict3/e53076d3-c3b7-47b8-8c63-30a08b8ad983___40213__Maruti_Suzuki_Swift--016.JPG.jpg'

display(Image(filename=image_path, width=600))

In [ ]:
!mkdir "/kaggle/working/runs/detect/EasyOCRFinal"

In [ ]:
#EasyOCR - Number plate character recognition

import os
from ultralytics import YOLO
import cv2
import easyocr
import numpy as np
from google.colab.patches import cv2_imshow

# Load YOLO11 model
model = YOLO('/kaggle/working/runs/detect/train2/weights/best.pt')

# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'])

# Define directories
input_dir = '/kaggle/working/ultralytics/datasets/test/'  # Directory with input images
output_dir = '/kaggle/working/runs/detect/EasyOCRFinal/'  # Directory to save annotated images

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def detect_and_recognize_batch(input_dir, output_dir):
    # Iterate over all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            # Process each image
            detect_and_recognize(image_path, output_path)

            # Display the original and predicted images side by side
            original_image = cv2.imread(image_path)
            predicted_image = cv2.imread(output_path)

            combined_image = np.hstack((original_image, predicted_image))  # Combine images horizontally
            cv2_imshow(combined_image)
            cv2.waitKey(1000)  # Display for 1 second (adjust as needed)

    cv2.destroyAllWindows()  # Close all OpenCV windows after processing

def detect_and_recognize(image_path, output_path):
    # Load image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Run YOLOv11 inference
    results = model.predict(source=image, save=False, save_txt=False)  # Perform inference
    detections = results[0].boxes.data  # Extract bounding box predictions

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection[:6]
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        # Crop the detected license plate
        plate_image = image[y1:y2, x1:x2]
        

        # Convert plate image to grayscale for OCR
        gray_plate = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)

        try:
            # Perform OCR on the cropped license plate
            ocr_result = reader.readtext(gray_plate, detail=1)  # Detailed output for bounding boxes

            if not ocr_result:
                print(f"No text detected in license plate region for image {image_path}")
                continue  # Skip to the next detection

            # Identify the maximum height of text boxes
            max_height = max(abs(box[0][1] - box[2][1]) for box, _, _ in ocr_result)

            # Filter out smaller text regions
            filtered_texts = []
            for box, text, _ in ocr_result:
                text_height = abs(box[0][1] - box[2][1])  # Calculate height of the current text box
                if text_height >= 0.5 * max_height:  # Include only texts with height >= 50% of max height
                    filtered_texts.append(text)

            # Combine filtered texts
            plate_text = " ".join(filtered_texts)

            if plate_text:  # Only annotate if valid text is found
                print(f"Detected Plate Text: {plate_text}")
                cv2.rectangle(original_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(original_image, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        except Exception as e:
            print(f"Error processing OCR for image {image_path}: {e}")
            continue  # Skip this region if OCR processing fails

    # Save the annotated image
    cv2.imwrite(output_path, original_image)
    print(f"Processed image saved to {output_path}")


# Run for the batch processing
#detect_and_recognize_batch(input_dir, output_dir)

# Run for individual image processing
image_path='/kaggle/working/ultralytics/datasets/val/PB19.jpg'
output_path='/kaggle/working/runs/detect/EasyOCRFinal/PB19_ocr.jpg'
detect_and_recognize(image_path, output_path)

display(Image(filename=output_path, width=600))

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def compare_images_side_by_side(path1, path2, top_n=10):
    # Get list of image files from both directories, excluding .txt files
    files1 = {os.path.splitext(f)[0]: f for f in os.listdir(path1) if os.path.isfile(os.path.join(path1, f)) and not f.endswith('.txt')}
    files2 = {os.path.splitext(f)[0]: f for f in os.listdir(path2) if os.path.isfile(os.path.join(path2, f)) and not f.endswith('.txt')}
    
    # Find filenames with at least 99% similarity
    matched_filenames = []
    for f1 in files1.keys():
        for f2 in files2.keys():
            if similar(f1, f2) > 0.99:
                matched_filenames.append((f1, f2))
    
    matched_filenames = matched_filenames[:top_n]  # Limit number of images to display
    
    if not matched_filenames:
        print("No similar images found between the directories.")
        return
    
    for f1, f2 in matched_filenames:
        img1_path = os.path.join(path1, files1[f1])
        img2_path = os.path.join(path2, files2[f2])
        
        img1 = cv2.imread(img1_path)
        img2 = cv2.imread(img2_path)
        
        if img1 is None or img2 is None:
            print(f"Error loading images: {img1_path} or {img2_path}")
            continue
        
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(img1)
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(img2)
        plt.axis('off')
        
        plt.show()
        print("-" * 50)

# Usage
compare_images_side_by_side(
    path1='/kaggle/working/ultralytics/datasets/val/',
    path2='/kaggle/working/runs/detect/EasyOCRFinal/',
    top_n=20  # You can change the value to display more or fewer images
)